In [15]:
from recongntionFacial import * 
# Main Code Setup

# base_dir = './train'

#     # image_height=112
    
#     # image_width=92
# images, labels = load_images(base_dir)
# images = images.reshape(len(images), image_height * image_width)
# images = images / 255.0
# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# pca = PCA(n_components=10)
# X_train_pca = pca.fit_transform(X_train)

# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(X_train_pca, y_train)

# X_test_pca = pca.transform(X_test)
# y_pred = knn.predict(X_test_pca)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Classification accuracy on the test set: {accuracy * 100:.2f}%")


In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import display, clear_output
import os

# Function to update the UI during training
def on_train_button_clicked(b):
    output.clear_output(wait=True)
    with output:
        print("Training started... Please wait.")
        train_model()

# Initialize the models and data as None or empty
knn = None
pca = None
X_train = None
y_train = None
X_train_pca = None

def train_model():
    global knn, pca, X_train, y_train, X_train_pca  # Declare as global to modify these variables outside this function
    base_dir = './train'  # Path to the training data folder (set directly in the code)
    if not os.path.isdir(base_dir):
        print("Please ensure the training folder exists.")
        return
    image_height=112
    
    image_width=92
    images, labels = load_images(base_dir)  # Load the images
    images = images.reshape(len(images), image_height * image_width)
    images = images / 255.0
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    # Fit PCA
    pca = PCA(n_components=10)
    X_train_pca = pca.fit_transform(X_train)
    
    # Fit KNN
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train_pca, y_train)
    
    # Test KNN
    X_test_pca = pca.transform(X_test)
    y_pred = knn.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Classification accuracy on the test set: {accuracy * 100:.2f}%")
    
    # Display scatter plot after fitting
    plot_pca_scatter(X_train, y_train, pca, show_plot=True, save_image=False)

def test_image(b):
    test_image_path = test_image_label.value.split(": ")[1]  # Get the file path from the label
    threshold_value = threshold_slider.value  # Get threshold value from the slider
    
    with test_output:
        clear_output(wait=True)
        print("Testing image...")
        print(f"Selected test image path: {test_image_path}")  # Print the file path
        print(f"Threshold value: {threshold_value}")  # Print the threshold value
        
        # Check if the models (knn, pca) are initialized
        if knn is None or pca is None:
            print("Error: Models are not initialized. Please train the model first.")
            return  # Exit if models are not initialized
        
        # Execute the prediction using the selected file path
        test_prediction(test_image_path, knn, pca, threshold=threshold_value, show_plot=show_faces_checkbox.value, save_image=save_images_checkbox.value)

# Button to trigger PCA and KNN fitting
fit_button = widgets.Button(description="Fit PCA & KNN")
fit_button.on_click(on_train_button_clicked)

# Output area to show training progress and accuracy
output = widgets.Output()

# Checkboxes for displaying options
show_scatter_checkbox = widgets.Checkbox(value=True, description="Show PCA Scatter")
show_faces_checkbox = widgets.Checkbox(value=True, description="Show Sample Faces")
save_images_checkbox = widgets.Checkbox(value=False, description="Save Images")

# Slider for setting dynamic threshold
threshold_slider = widgets.FloatSlider(
    value=3.5,
    min=0.0,
    max=9.99,
    step=0.01,
    description="Threshold",
    style={'description_width': 'initial'}
)

# File upload button for selecting a test image file
test_image_button = widgets.FileUpload(
    description="Choose Test Image",
    accept='.pgm',  # Only accept .pgm files (can be adjusted)
    multiple=False
)

# Output area for displaying the result of the test image prediction
test_output = widgets.Output()

# Path to temporarily save the selected file
upload_dir = './uploads'
os.makedirs(upload_dir, exist_ok=True)

# The function to handle the file selection
def on_test_image_button_uploaded(change):
    uploaded_file = list(test_image_button.value.keys())[0]
    test_image_path = os.path.join(upload_dir, uploaded_file)
    with open(test_image_path, 'wb') as f:
        f.write(test_image_button.value[uploaded_file]['content'])
    
    test_image_label.value = f"Selected File Path: {test_image_path}"
    print(f"Selected file path: {test_image_path}")

# Set up the file chooser button observer
test_image_button.observe(on_test_image_button_uploaded, names='value')

# Create a label to display the selected file path
test_image_label = widgets.Label(value="Selected File: None")

# Button to trigger testing
test_button = widgets.Button(description="Test Image")
test_button.on_click(test_image)

# Arrange the widgets for display with adjusted height and top margin
train_widgets = VBox([
    fit_button,
    output
], layout=Layout( margin='100px 0 0 0'))  # Adding margin-top of 100px

test_widgets = VBox([
    show_scatter_checkbox,
    show_faces_checkbox,
    save_images_checkbox,
    threshold_slider,
    test_image_label,  # Displaying the selected file path
    test_image_button,  # File chooser button
    test_button,  # Button to trigger testing
    test_output
], layout=Layout(margin='100px 100px 200px 100px'))  # Adding margin-top of 100px

# Display all widgets
display(VBox([train_widgets, test_widgets]))


Selected file path: ./uploads/10.pgm


Selected file path: ./uploads/10.pgm
